In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NewsTechLSTM(nn.Module):
    def __init__(self, embedding_dim=100, cnn_out_channels=64, news_hidden=128, tech_hidden=64,dropout=0.5, num_classes=2):
        super(NewsTechLSTM, self).__init__()

        # CNN + LSTM for News
        self.conv1 = nn.Conv1d(embedding_dim, cnn_out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.dropout = nn.Dropout(dropout)
        self.news_lstm = nn.LSTM(cnn_out_channels, news_hidden, batch_first=True)

        # LSTM for Technical Indicators
        self.tech_lstm = nn.LSTM(7, tech_hidden, batch_first=True)

        # Classifier
        self.fc = nn.Linear(news_hidden + tech_hidden, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, news_seq, tech_seq):
        x = news_seq.permute(0, 2, 1) 
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = x.permute(0, 2, 1)

        news_out, _ = self.news_lstm(x) 
        news_last = news_out[:, -1, :]  
        tech_out, _ = self.tech_lstm(tech_seq)
        tech_last = tech_out[:, -1, :] 

        combined = torch.cat((news_last,tech_last),dim=1)
        out=self.fc(combined)
        return self.softmax(out)